# This is a **cleared** version of notebook used during data preparation. Something may not work!

In [1]:
import requests
import time

from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import undetected_chromedriver as uc

import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser

from iteration_utilities import duplicates
import imageio.v3 as iio
import tqdm

import re
import pickle

In [2]:
BASE = 'https://anilist.co'
NOT_FOUND = '/img/404/404_chan1.jpg'
HEADERS = {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'}

In [3]:
def init_driver():
    u = 'https://anilist.co/anime/1'
    
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    #chrome_options.add_argument("--headless")
    #chrome_options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 2})

    #driver = uc.Chrome(options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)

    driver.implicitly_wait(10)

    driver.execute_cdp_cmd('Network.setUserAgentOverride', HEADERS)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    driver.get(u)
    time.sleep(3)

    #try:
    #    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[1]/div/div/div/div[2]/div/button[2]'''))).click()
    #except Exception as e:
    #    print(e)
    
    return driver

In [4]:
driver = init_driver()

In [5]:
with open('normal_links.txt', 'r') as f:
    links = f.read().split()

In [6]:
with open('adult_links.txt', 'r') as f:
    l = f.read().split()

In [7]:
def parse_links(driver, links):

    data = {}
    
    for link in tqdm.tqdm(links):

        # load and check page
        page = BASE + link
        driver.get(page)
        driver.implicitly_wait(2)
        if NOT_FOUND in driver.page_source: continue

        # get id
        pattern = re.search('''anime\/([0-9]+)''', link)
        idx = pattern.group(1)

        # click spoiler toggle
        try:
            btn = driver.find_element(By.CLASS_NAME, "spoiler-toggle")
            action = webdriver.ActionChains(driver)
            action.move_to_element(btn)

            try:
                WebDriverWait(driver, 3).until(EC.element_to_be_clickable(btn)).click()

                # unfocus tags panel
                element = driver.find_element(By.CLASS_NAME, 'review button edit')
                action.move_to_element(element)
            except ElementClickInterceptedException as e:
                print(e)
        except NoSuchElementException as e:
            pass

        data[idx] = driver.page_source
        
    return data

In [8]:
def parse(min_v, max_v, links, step=1000):
    for i in range(min_v, max_v, step):
        data = parse_links(driver, links[i:i+step])

        with open(f'from_{i}_to_{i+step-1}.pickle', 'wb') as f:
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# parsing separated to chunks due to problems with internet connection
parse(0, len(links+l)+1, links+l, step=1000)

In [9]:
parse(9000, len(links+l)+1, links+l, step=1000)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [10:15<00:00,  2.46s/it]


In [12]:
with open('from_7000_to_7999.pickle', 'rb') as f:
    d7000 = pickle.load(f)

In [14]:
list(d7000.keys())[700:710]

['10501',
 '17697',
 '9088',
 '10802',
 '10624',
 '17233',
 '10301',
 '102303',
 '10629',
 '6919']

In [25]:
with open('tmp.html', 'w', encoding='utf-8') as f:
    f.write(d7000['6919'])